# Image combination

Imge combination serves a few purposes. Combining images

+ reduces noise in images.
+ can remove transient artifacts like cosmic rays and satellite tracks.
+ can remove stars in twilight flats from the combined image.

It is essential that several of each type of calibration image (bias, dark, flat) be taken. Combining them reduces the noise in the images by roughly a factor of $1/\sqrt{N}$, where $N$ is the number of images being combined. As shown in the previous notebook, using a single calibration image actually *increases* the noise in your image.

There are a few ways to combine images; if done properly, features that show up in only one of the images (like cosmic rays) are not present in the combination. Done incorrectly, those features show up in your combined images and then contaminate your calibrated science images too.

### The bottom line: combine by averaging images, but clip extreme values

The remainder of the notebook motivates that conclusion and explains how to do that combination with [ccdproc](https://ccdproc.readthedocs.io/en/latest/).

In [ ]:
import numpy as np

%matplotlib inline
from matplotlib import pyplot as plt
from matplotlib import rc

from astropy.visualization import hist

In [ ]:
# Set some default parameters for the plots below
rc('font', size=20)
rc('axes', grid=True)

## Combination method: average or median?

In this section we'll look at a simplified version of the problem one faces in combining images to reduce noise. It is fair to think of astronomical images (especially bias and dark images) as being a Gaussian distribution of pixel values around the bias level, and a wdith related to the read noise of the detector. In properly done flat images the noise is technically Poisson distribution, but with a large enough number of counts that the distribution is indistringuishable from a Gaussian distribution whose width is related the square root of the number of counts. While some regions of a science image are dominated by Poisson noise from sources in the image, most of the image will be dominated by Gaussian read noise from the detector or Poisson noise from the sky background.

Instead of working with a combination of images, we'll create 100 Gaussian distributions with mean of zero, and standard deviation 1 and combine those two different ways: by finding the average and by finding the median. Each distribution has size $320^2$ so that we can easily view it as either a distribution of 102,400 values or as an image that is $320 \times 320$.

You should think of each of these 100 distributions as representing an image, like a bias or dark. To make the analogy to real images a little more direct, a "bias" of 1000 is added to each distribution.

In [ ]:
n_distributions = 100
bias_level = 1000
n_side = 320
bits = np.random.randn(n_distributions, n_side**2) + bias_level
average = np.average(bits, axis=0)
median = np.median(bits, axis=0)

Now that we've created the distributions and combined them in two different ways, let's take a look at them. The [`hist` function from astropy.visualization](https://astropy.readthedocs.io/en/stable/visualization/histogram.html) is used below because it can figure out what bin size to use for your data. *Note: but beware https://github.com/astropy/astropy/issues/7758*

In [ ]:
fig, ax = plt.subplots(1, 2, sharey=True, tight_layout=True, figsize=(20, 10))

hist(bits[0, :], bins='freedman', ax=ax[0]);
ax[0].set_title('One sample distribution')

hist(average, bins='freedman', label='average', alpha=0.5, ax=ax[1]);
hist(median, bins='freedman', label='median', alpha=0.5, ax=ax[1]);
ax[1].set_title('{} distributions combined'.format(n_distributions))
ax[1].legend()

Combining by averaging clearly gives a narrower (i.e. less noisy) distribution than combining by median, though both substantially reduced the width of the distribution. The conclusion so far is that combining by averaging is mildly preferable to combining by median. Computationally, the mean is also faster compute than the median.

#### Image view of these distributions

As suggested above, we could view each of these distributions as an image instead of a histogram. One take away from the diagram below is that in this case the difference between mean and median is not at all apparent.

In all cases the extreme values of the image display are set to bracket the width of the initial distribution.

In [ ]:
fig, axes = plt.subplots(1, 3, sharey=True, tight_layout=True, figsize=(20, 10))
data_source = [bits[0, :], average, median]
titles = ['One distrbution', 'Average', 'Median']

for axis, data, title in zip(axes, data_source, titles):
    axis.imshow(data.reshape(n_side, n_side), vmin=bias_level - 3, vmax=bias_level + 3)
    axis.set_xticks([])
    axis.set_yticks([])
    axis.grid(False)
    axis.set_title(title)

### The effect of outliers

Suppose that, in just one of the 100 distributions we are combining, there are a small number of extreme values. In astronomical images these extremes happen very frequently because of cosmic ray hits on the detector that cause, in one small patch of a calibration image, much higher counts. Another case is when combining twilight flats, which often contain faint images of stars.

In the example below we set just 50 points out of the 102,400 in the first distribution to a somewhat higher value than the rest. 

In [ ]:
bits[0, 10000:10050] = 2 * bias_level

In [ ]:
plt.imshow(bits[0, :].reshape(n_side, n_side), vmin=bias_level - 3, vmax=bias_level + 3)
plt.xticks([])
plt.yticks([])
plt.grid(False)

In [ ]:
average = np.average(bits, axis=0)
median = np.median(bits, axis=0)

In [ ]:
hist(average, bins='freedman', alpha=0.5);
hist(median, bins='freedman', alpha=0.5);
plt.semilogy();

In [ ]:
plt.imshow(average.reshape(n_side, n_side), vmin=bias_level - 3, vmax=bias_level + 3)
plt.xticks([])
plt.yticks([])
plt.grid(False)

In [ ]:
plt.imshow(median.reshape(n_side, n_side), vmin=bias_level - 3, vmax=bias_level + 3)
plt.grid(False)

In [ ]:
plt.imshow(bits[1, :].reshape(n_side, n_side), vmin=bias_level - 3, vmax=bias_level + 3)
plt.grid(False)